---

In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
import sys
sys.path.append(sys.path[0].replace('notebooks', 'src'))

import modeling.modeling_utils as m
from modeling.deepemo_utils import make_df

In [40]:
m.torch.__version__

'1.6.0'

In [41]:
m.torch.cuda.is_available() # Check for GPU

False

In [42]:
device = m.torch.device("cuda" if m.torch.cuda.is_available() else "cpu")

In [43]:
m.torch.backends.cudnn.benchmark=True # Helps optimize training w/ GPU

---

This notebook contains the code used to fine-tune a ```Resnet50``` model on the ```Deep Emotion``` dataset. In the context of this project, this model is tested against the curriculum-based ```WEBEmo``` trained model in the ```experiment``` notebook.

The ```Deep Emotion``` dataset can be downloaded here: https://www.cs.rochester.edu/u/qyou/deepemotion/. After downloading, unzip into the ```data``` directory.

# Load Data

In [ ]:
deepemo_df = make_df('../data/Flickr')
deepemo_df.head()

You should obtain a dataframe in this format:
```
file | labels
0 | excitement/excitement_1556.jpg | 3
1 | excitement/excitement_0890.jpg | 3
2 | excitement/excitement_0648.jpg | 3
3 | excitement/excitement_1230.jpg | 3
4 | excitement/excitement_2739.jpg | 3
```

Following that, the data are ready to be put into a Pytorch-friendly format and trained.

In [ ]:
# Separating into train, validation, and test splits
train, test = m.train_val_split(X=deepemo_df['file'], 
                                y=deepemo_df['labels'], 
                                test_size=0.2, 
                                random_state=713)
train_split, val_split = m.train_val_split(X=train_df['file'], 
                                           y=train_df['labels'], 
                                           test_size = 0.1, 
                                           random_state=611)

In [ ]:
train_transforms = m.transforms.Compose([m.transforms.Resize(256),
                                         m.transforms.RandomCrop(224),
                                         m.transforms.ToTensor(),
                                         m.transforms.Normalize(mean=[0.485, 0.456, 0.406], #OG means/sds from imagenet
                                                                std=[0.229, 0.224, 0.225])
                                        ])
val_transforms = m.transforms.Compose([m.transforms.Resize(256),
                                       m.transforms.CenterCrop(224),
                                       m.transforms.ToTensor(),
                                       m.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])
                                      ])

In [32]:
train_dataset = m.ImgDataset(df=train_split,
                             root_dir='../data/Flickr',
                             percent_sample=1,
                             transform=train_transforms)
val_dataset = m.ImgDataset(df=val_split,
                           root_dir='../data/Flickr',
                           percent_sample=1,
                           transform=val_transforms)

In [77]:
samples_weights = m.weighted_sample(train_split, 'labels')
weighted_sampler = m.WeightedRandomSampler(weights=samples_weights, num_samples=len(samples_weights))

In [35]:
# Pytorch DataLoader iteratively loads minibatches during training
loaders = {'train': m.DataLoader(train_dataset,
                                 batch_size=128, 
                                 sampler=weighted_sampler,
                                 #pin_memory=True, # Only use pin_memory with GPU
                                 num_workers=4), 
           'val': m.DataLoader(val_dataset, 
                               batch_size=128,
                               #pin_memory=True,
                               num_workers=4)}

# Train Model

Similar to the level-1 ```WEBEmo``` model, a pre-trained ```Resnet50``` is fine-tuned on the ```Deep Emotion``` dataset.

In [56]:
model = m.models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = m.nn.Linear(num_ftrs, 8) # 8 possible emotions
model = model.to(device)
criterion = m.nn.CrossEntropyLoss().to(device)
optim = m.torch.optim.SGD(model.parameters(), 
                          lr=0.001, 
                          momentum=0.9, 
                          weight_decay=0.0001)
scheduler = m.lr_scheduler.ReduceLROnPlateau(optim, patience=5)

In [ ]:
# Train
model_train = m.train_model(model=model, 
                            dataloader=loaders_dict, 
                            criterion= criterion, 
                            optimizer= optim,
                            save_path='../models/deepemo_model.tar',
                            num_epochs=50,
                            scheduler= scheduler,
                            early_stopping=m.EarlyStopping(patience=10))

# Evaluate on Test Set

In [ ]:
test_dataset = m.ImgDataset(df=test,
                            root_dir='../data/Flickr',
                            percent_sample=1,
                            transform=val_transforms)
test_loader = m.DataLoader(test_dataset, 
                           batch_size=128,
                           #pin_memory=True,
                           num_workers=4)

In [ ]:
test_model = m.load_model(path='../models/deepemo_model.tar', 
                          base=m.models.resnet50(pretrained=False), 
                          old_classes=8, 
                          new_classes=8, 
                          device=device)
test_model = test_model.eval()
test_model = test_model.to(device)

In [ ]:
m.evaluate_model(test_model, test_loader)